In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats
from scipy.stats import pearsonr
import itertools
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

In [93]:
train_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
oil_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
transaction_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
stores_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
holiday_event_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')


train_df = pd.read_csv('/kaggle/working/ml_train.csv')

In [94]:
train_df.columns

Index(['Unnamed: 0', 'date', 'sales', 'onpromotion', 'diff_sales'], dtype='object')

In [95]:
train_df

,Unnamed: 0,date,sales,onpromotion,diff_sales
0,4,2013-01-05,477350.12123,0,122890.44414
1,5,2013-01-06,519695.40109,0,42345.27986
2,6,2013-01-07,336122.80107,0,-183572.60002
3,7,2013-01-08,318347.77798,0,-17775.02308
4,8,2013-01-09,302530.80902,0,-15816.96896
...,...,...,...,...,...
1675,1679,2017-08-11,826373.72202,14179,174986.81005
1676,1680,2017-08-12,792630.53508,8312,-33743.18694
1677,1681,2017-08-13,865639.67747,9283,73009.14239
1678,1682,2017-08-14,760922.40608,8043,-104717.27139


In [114]:
# Define the hyperparameters for the LGBMRegressor model
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mean_absolute_percentage_error',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'max_depth': 10,
    'min_split_gain':0.0
}

In [115]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
features = ['diff_sales', 'onpromotion']

# Splitting into features (X) and target (y)
X = train_df[features]
y = train_df['sales']

# Define the number of splits for cross-validation (e.g., 5-fold cross-validation)
n_splits = 10

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)
mapelist=[]
# Perform cross-validation
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the LightGBM model
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    
    # Evaluate the model using MAPE
    mape = mean_absolute_percentage_error(y_test, predictions)
    mapelist.append(mape)
print("Mean Absolute Percentage Error:", mape.min())

Mean Absolute Percentage Error: 0.11031580004323382


### ML with these hyperparameters didn't have a better mape than ARIMA. Experimentation should be investigated further with different hyperparameters.